In [1]:
import pandas as pd
from openai import OpenAI
import time
import requests
import scispacy
import spacy
from scispacy.linking import EntityLinker
from spacy.language import Language
from bs4 import BeautifulSoup

**Descrição:**
A ideia deste notebook é fazer uma análise exploratória das bases do tipo disease-symptom que eu mapeei, são caracterizada por uma coluna para cada sintoma (0 ou 1) e uma coluna com o prognóstico da doença. O objetivo é ver a quantidade de sintomas e pesquisar a melhor forma de unir as bases, mantendo as características originais e evitando doenças com poucos casos. Após isso, serão estudadas as formas de junção!

## Dataset 1 - Kaggle

**Descrição:** O conjunto de dados contém nomes de doenças juntamente com os sintomas apresentados pelo respectivo paciente. Há um total de 773 doenças únicas e 377 sintomas, com aproximadamente 246.000 linhas. O conjunto de dados foi gerado artificialmente, preservando a severidade dos sintomas e a probabilidade de ocorrência das doenças.

**Informações de Interpretação:** Vários grupos distintos de sintomas podem ser indicadores de uma mesma doença. Pode até haver um único sintoma contribuindo para a identificação de uma doença em uma linha ou amostra. Isso é um indicativo de alta correlação entre o sintoma e aquela doença específica.
Um número maior de linhas para uma determinada doença corresponde a uma maior probabilidade de ocorrência no mundo real. Da mesma forma, em uma linha, se o vetor de características contém apenas um sintoma, isso implica que esse sintoma possui uma correlação mais forte para classificar a doença do que qualquer sintoma presente em um vetor com múltiplos sintomas em outra amostra.

**Referência:** https://www.kaggle.com/datasets/dhivyeshrk/diseases-and-symptoms-dataset

In [2]:
# Importando o primeiro dataset para análise, esse é o mais robusto
dataset_kaggle = pd.read_csv('./datasets/Final_Augmented_dataset_Diseases_and_Symptoms.csv')
dataset_kaggle.head()

,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,panic disorder,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,panic disorder,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# 246.945 casos, com 377 sintomas mapeados
dataset_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246945 entries, 0 to 246944
Columns: 378 entries, diseases to neck weakness
dtypes: int64(377), object(1)
memory usage: 712.2+ MB


In [4]:
#Contagem de quantos casos de cada doença apresentaram determinado sintoma, são 773 doenças mapeadas e 377 sintomas
dataset_kaggle.groupby('diseases').sum()

,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,palpitations,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
diseases,,,,,,,,,,,,,,,,,,,,,
abdominal aortic aneurysm,0,0,105,0,0,0,0,0,0,110,...,0,0,0,0,0,0,0,0,0,0
abdominal hernia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abscess of nose,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abscess of the lung,0,0,17,19,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abscess of the pharynx,0,0,0,0,266,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
white blood cell disease,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
whooping cough,0,0,31,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wilson disease,0,0,0,0,0,0,14,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Checando se existem valores nulos, pelo visto não
valores_nulos = dataset_kaggle.isnull().sum()
valores_nulos

diseases                            0
anxiety and nervousness             0
depression                          0
shortness of breath                 0
depressive or psychotic symptoms    0
                                   ..
hip weakness                        0
back swelling                       0
ankle stiffness or tightness        0
ankle weakness                      0
neck weakness                       0
Length: 378, dtype: int64

In [6]:
valores_nulos.sum() #tem mesmo não

0

In [7]:
# Contagem das doenças para verificar as mais frequentes
# É interessante retirar as doenças menos frequentes antes de mesclar as bases?
dataset_kaggle['diseases'].value_counts()

diseases
cystitis                          1219
vulvodynia                        1218
nose disorder                     1218
complex regional pain syndrome    1217
spondylosis                       1216
                                  ... 
typhoid fever                        1
rocky mountain spotted fever         1
open wound of the knee               1
hypergammaglobulinemia               1
open wound due to trauma             1
Name: count, Length: 773, dtype: int64

## Dataset 2 - Symbi Predict

**Descrição:** O Symptom-Disease Prediction Dataset (SDPD) é uma coleção abrangente de dados estruturados que relacionam sintomas a diversas doenças, meticulosamente curada para facilitar pesquisas e o desenvolvimento de análises preditivas em saúde. Inspirado na metodologia empregada por instituições renomadas como os Centers for Disease Control and Prevention (CDC), este conjunto de dados tem como objetivo fornecer uma base confiável para o desenvolvimento de modelos de predição de doenças baseados em sintomas.O dataset abrange uma ampla variedade de sintomas, extraídos de literatura médica confiável, observações clínicas e consenso de especialistas.

**Referência:** https://data.mendeley.com/datasets/dv5z3v2xyd/1

In [8]:
dataset_symbi = pd.read_csv('./datasets/symbipredict_2022.csv')
dataset_symbi.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection


In [9]:
# 132 sintomas, mapeados em 4961 casos (menor que o primeiro)
dataset_symbi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4961 entries, 0 to 4960
Columns: 133 entries, itching to prognosis
dtypes: int64(132), object(1)
memory usage: 5.0+ MB


In [10]:
#sem valores nulos também, o nome dos sintomas é escrito de maneira diferente, é bom observar isso na hora da junção
dataset_symbi.isnull().sum()

itching                 0
skin_rash               0
nodal_skin_eruptions    0
continuous_sneezing     0
shivering               0
                       ..
inflammatory_nails      0
blister                 0
red_sore_around_nose    0
yellow_crust_ooze       0
prognosis               0
Length: 133, dtype: int64

In [11]:
dataset_symbi.groupby('prognosis').sum()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
prognosis,,,,,,,,,,,,,,,,,,,,,
AIDS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acne,0,115,0,0,0,0,0,0,0,0,...,109,109,109,0,0,0,0,0,0,0
Alcoholic Hepatitis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Allergy,0,0,0,109,109,109,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arthritis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bronchial Asthma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cervical Spondylosis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chickenpox,115,115,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Chronic Cholestasis,115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Contagem das doenças para verificar as mais frequentes
# Todas possuem 121 casos
dataset_symbi['prognosis'].value_counts()

prognosis
Fungal Infection                 121
Hepatitis C                      121
Hepatitis E                      121
Alcoholic Hepatitis              121
Tuberculosis                     121
Common Cold                      121
Pneumonia                        121
Dimorphic Hemmorhoids (piles)    121
Heart Attack                     121
Varicose Veins                   121
Hypothyroidism                   121
Hyperthyroidism                  121
Hypoglycemia                     121
Osteoarthritis                   121
Arthritis                        121
Vertigo                          121
Acne                             121
Urinary Tract Infection          121
Psoriasis                        121
Hepatitis D                      121
Hepatitis B                      121
Allergy                          121
Hepatitis A                      121
GERD                             121
Chronic Cholestasis              121
Drug Reaction                    121
Peptic Ulcer Disease        

In [13]:
#vamos padronizar o df de acordo com o primeiro, a coluna diseases sendo a primeira e com espaço ao invés de _ nas colunas
dataset_symbi = dataset_symbi.rename(columns=lambda x: x.replace("_", " "))
col_prognosis = dataset_symbi.pop('prognosis')
dataset_symbi.insert(0, 'diseases', col_prognosis)
dataset_symbi.head()


,diseases,itching,skin rash,nodal skin eruptions,continuous sneezing,shivering,chills,joint pain,stomach pain,acidity,...,pus filled pimples,blackheads,scurring,skin peeling,silver like dusting,small dents in nails,inflammatory nails,blister,red sore around nose,yellow crust ooze
0,Fungal Infection,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fungal Infection,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Fungal Infection,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fungal Infection,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fungal Infection,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Agora que já fiz duas, observei coisas importantes e tive algumas ideias:

1.   Na hora de junção, é impotante comparar os nomes de doenças e sintomas para prevenir duplicatas
2.   Seria interessante somar os sintomas por doença e atribuir um peso aos valores obtidos?
3.   E se eu fornecer ao modelo uma descrição para cada doença da base e formas de tratamento para gerar uma resposta precisa e assertiva?

## Dataset 3 - Columbia

**Observação:** Este dataset é o resultado de um processo de tratamento e limpeza feito por um usuário do kaggle a partir de um dataset da Universidade de Columbia, conta com 134 doenças e 405 sintomas. 

**Descrição do dataset original no site da universidade:** A tabela  representa uma base de conhecimento com associações entre doenças e sintomas, gerada por um método automatizado com base em informações extraídas de sumários de alta hospitalar em formato textual de pacientes do Hospital Presbiteriano de Nova York, admitidos durante o ano de 2004. As associações foram calculadas para as 150 doenças mais frequentes, com base nessas notas clínicas, e os sintomas são apresentados em ordem de força da associação. O método utilizou o sistema de processamento de linguagem natural MedLEE para extrair códigos UMLS de doenças e sintomas a partir dos textos. Posteriormente, métodos estatísticos baseados em frequências e coocorrências foram empregados para calcular as associações. Uma descrição mais detalhada do método automatizado pode ser encontrada em: Wang X, Chused A, Elhadad N, Friedman C, Markatou M. Automated knowledge acquisition from clinical reports. AMIA Annu Symp Proc. 2008. p. 783-7. PMCID: PMC2656103.

**Referências:**

> Dataset original: https://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/

> Dataset tratado do kaggle: https://www.kaggle.com/datasets/wisnuafifuddin/disease-symptom-knowledge-database-by-columbia-edu


In [14]:
dataset_columbia = pd.read_csv('./datasets/fixed_augmented_dataset_multibiner_num_augmentations_100_cleaned.csv')
dataset_columbia.head()

,yellow sputum,cardiovascular finding,hypercapnia,heavy feeling,ambidexterity,polymyalgia,stinging sensation,shortness of breath,palpitation,hypokalemia,...,pain,numbness of hand,Murphy's sign,air fluid level,muscle hypotonia,cough,weight gain,hot flush,blackout,prognosis
0,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,hypertensive disease
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,diabetes
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,depression mental
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,coronary arteriosclerosis
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,pneumonia


In [15]:
#405 sintomas, mapeados em 11703 casos
dataset_columbia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11703 entries, 0 to 11702
Columns: 406 entries, yellow sputum to prognosis
dtypes: int64(405), object(1)
memory usage: 36.3+ MB


In [16]:
#sem nulos por aqui também
dataset_columbia.isnull().sum()

yellow sputum             0
cardiovascular finding    0
hypercapnia               0
heavy feeling             0
ambidexterity             0
                         ..
cough                     0
weight gain               0
hot flush                 0
blackout                  0
prognosis                 0
Length: 406, dtype: int64

In [17]:
#133 doenças mapeadas
dataset_columbia.groupby('prognosis').sum()

,yellow sputum,cardiovascular finding,hypercapnia,heavy feeling,ambidexterity,polymyalgia,stinging sensation,shortness of breath,palpitation,hypokalemia,...,para 2,pain,numbness of hand,Murphy's sign,air fluid level,muscle hypotonia,cough,weight gain,hot flush,blackout
prognosis,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,50,0,0,0
Pneumocystis carinii pneumonia,46,0,0,0,0,0,0,0,0,40,...,0,0,0,0,0,0,0,0,0,0
accident cerebrovascular,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acquired immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,45,49,0,0,0
adenocarcinoma,0,0,0,0,0,0,0,0,0,0,...,0,41,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic epilepsy,0,0,0,0,35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
transient ischemic attack,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tricuspid valve insufficiency,0,0,0,0,0,0,0,48,0,0,...,0,44,0,0,0,0,0,0,0,0


In [18]:
#esse aqui não tá distribuído de maneira homogênea, mas o menor valor tem 8 casos (decubitus ulcer)
dataset_columbia['prognosis'].value_counts()

prognosis
malignant neoplasms         178
psychotic disorder          100
candidiasis                  99
emphysema pulmonary          99
anxiety state                99
                           ... 
aphasia                      62
migraine disorders           60
failure heart congestive     59
kidney disease               42
decubitus ulcer               8
Name: count, Length: 133, dtype: int64

In [19]:
col_prognosis = dataset_columbia.pop('prognosis')
dataset_columbia.insert(0, 'diseases', col_prognosis)
dataset_columbia.head()

,diseases,yellow sputum,cardiovascular finding,hypercapnia,heavy feeling,ambidexterity,polymyalgia,stinging sensation,shortness of breath,palpitation,...,para 2,pain,numbness of hand,Murphy's sign,air fluid level,muscle hypotonia,cough,weight gain,hot flush,blackout
0,hypertensive disease,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,diabetes,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,depression mental,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,coronary arteriosclerosis,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,pneumonia,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


## Técnicas de junção


**Sugestões:**

1.   Utilizar técnicas fuzzy de junção de texto para capturar as similaridades entre os nomes de doenças e sintomas, já que existem sintomas iguais com nomes diferentes nas bases
2.   Somar as colunas de sintoma para descobrir quantas vezes um sintoma aparece por caso e adicionar um peso a isso
3.   Aplicar técnicas de oversampling ou undersampling para balancear os casos por doença
4. Enriquecer os dados com fontes externas para melhorar a precisão do modelo



### Normalização e Padronização

O primeiro passo vai ser normalizar e padronizar os nomes das doenças e sintomas para evitar duplicatas, garantindo a consistência dos dados no treinamento. Lib utilizada: https://github.com/allenai/scispacy

In [20]:
# Carregar o modelo do SciSpaCy
nlp = spacy.load("en_core_sci_md")  # Ou "en_core_sci_lg" para mais precisão
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

c:\Users\mario\.conda\envs\medical_llm\lib\site-packages\spacy\language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
c:\Users\mario\.conda\envs\medical_llm\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mario\.conda\envs\medical_llm\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#secu

In [41]:
#esta função aplica o nlp no nome da doença para padronizar os valores de acordo com o UMLS
def get_condiction_name(name):
    doc = nlp(name)

    if not doc.ents:
        return name  #retorna o nome original se nenhuma entidade for detectada

    entity = doc.ents[0]  #pega a primeira entidade detectada

    #acessar o linker
    linker = nlp.get_pipe("scispacy_linker")

    #se a entidade tem links no UMLS, retorna o nome do conceito mais relevante
    if hasattr(entity._, "kb_ents") and entity._.kb_ents:
        umls_id = entity._.kb_ents[0][0]  #pega o UMLS ID de maior score
        disease_entity = linker.kb.cui_to_entity[umls_id] #pega a entidade da condição no umls (Unified Medical Language System)
        disease_name = disease_entity.canonical_name
        if len(disease_name.split()) < len(name.split()):
            return name
        return disease_name #retorna o nome da doença para substituir na base

    return entity.text  #se não houver links, retorna o nome detectado

#testando a função
get_condiction_name("heart attack")


'Myocardial Infarction'

In [22]:
from difflib import SequenceMatcher
# Fallback manual para sintomas comuns
manual_fallback = {
    "abdominal": "abdominal pain",
    "head": "headache",
    "chest": "chest pain",
    "back": "back pain",
    "feverish": "fever",
    "tired": "fatigue",
}

def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def get_symptom_name(symptom, min_similarity=0.6):
    doc = nlp(symptom)
    
    if not doc.ents:
        return manual_fallback.get(symptom.lower(), symptom)  # fallback ou original

    # Seleciona a entidade mais longa (mais próxima do termo completo)
    best_entity = max(doc.ents, key=lambda ent: len(ent.text.split()))

    #acessar o linker
    linker = nlp.get_pipe("scispacy_linker")
    
    if hasattr(best_entity._, "kb_ents") and best_entity._.kb_ents:
        umls_id = best_entity._.kb_ents[0][0]
        umls_entity = linker.kb.cui_to_entity[umls_id]
        canonical = umls_entity.canonical_name

        # Se a similaridade for razoável ou o termo original for curto, retorna padronizado
        if similarity(symptom, canonical) >= min_similarity or len(symptom.split()) < 2:
            return canonical

    # Se não houver entidade UMLS compatível ou baixa similaridade, tenta fallback ou retorna original
    return manual_fallback.get(symptom.lower(), symptom)

In [23]:
#função para atualizar as colunas com os nomes dos sintomas
def update_symptoms(df: pd.DataFrame) -> pd.DataFrame:
  new_df = df.copy()
  for column in new_df.columns[1:]:
    new_symptom_name = get_symptom_name(column)
    if len(new_symptom_name.split()) < len(column.split()):
      new_symptom_name = column
    print(f'{column} --> {new_symptom_name}')
    new_df.rename(columns={column: new_symptom_name}, inplace=True)
  return new_df

In [24]:
#atualizando as bases para padronizar os nomes das doenças antes da junção
#coletar a lista única de doenças nas 4 bases
unique_diseases = set()

#adicionar as doenças de cada dataset na lista única
for dataset in [dataset_kaggle, dataset_symbi, dataset_columbia]:
    unique_diseases.update(dataset['diseases'].unique())

#padronizar os nomes das doenças
disease_mapping = {disease: get_condiction_name(disease) for disease in unique_diseases}

#substituir as doenças padronizadas nas bases de dados
dataset_kaggle['diseases'] = dataset_kaggle['diseases'].map(disease_mapping).fillna(dataset_kaggle['diseases'])
dataset_symbi['diseases'] = dataset_symbi['diseases'].map(disease_mapping).fillna(dataset_symbi['diseases'])
dataset_columbia['diseases'] = dataset_columbia['diseases'].map(disease_mapping).fillna(dataset_columbia['diseases'])

In [25]:
#Mapeamento das doenças padronizadas
disease_mapping

{'neoplasm': 'Neoplasms',
 'pregnancy': 'Pregnancy',
 'amyloidosis': 'Amyloidosis',
 'open wound of the chest': 'open wound of the chest',
 'goiter': 'Goiter',
 'postoperative infection': 'Postoperative infection',
 'foreign body in the ear': 'foreign body in the ear',
 'torticollis': 'Torticollis',
 'premature ventricular contractions (pvcs)': 'premature ventricular contractions (pvcs)',
 'carbon monoxide poisoning': 'carbon monoxide poisoning',
 'open wound of the head': 'open wound of the head',
 'tonsillar hypertrophy': 'Hypertrophy of tonsils',
 'mitral valve insufficiency': 'Mitral Valve Insufficiency',
 'vaginitis': 'Vaginitis',
 'diabetic retinopathy': 'Diabetic Retinopathy',
 'sialoadenitis': 'Sialadenitis',
 'foreign body in the eye': 'foreign body in the eye',
 'dysthymic disorder': 'Dysthymic Disorder',
 'problem during pregnancy': 'problem during pregnancy',
 'Allergy': 'Allergy Specialty',
 'trichinosis': 'Trichinellosis',
 'erectile dysfunction': 'Erectile dysfunction',


In [26]:
#atualizando as bases para padronizar os nomes dos sintomas antes da junção
dataset_kaggle = update_symptoms(dataset_kaggle)
dataset_symbi = update_symptoms(dataset_symbi)
dataset_columbia = update_symptoms(dataset_columbia)

anxiety and nervousness --> anxiety and nervousness
depression --> Mental Depression
shortness of breath --> shortness of breath
depressive or psychotic symptoms --> depressive or psychotic symptoms
sharp chest pain --> sharp chest pain
dizziness --> Dizziness
insomnia --> Sleeplessness
abnormal involuntary movements --> abnormal involuntary movements
chest tightness --> Chest tightness
palpitations --> Palpitations
irregular heartbeat --> irregular heartbeat
breathing fast --> breathing fast
hoarse voice --> hoarse voice
sore throat --> sore throat
difficulty speaking --> difficulty speaking
cough --> cough
nasal congestion --> Nasal congestion (finding)
throat swelling --> throat swelling
diminished hearing --> diminished hearing
lump in throat --> lump in throat
throat feels tight --> throat feels tight
difficulty in swallowing --> difficulty in swallowing
skin swelling --> skin swelling
retention of urine --> retention of urine
groin mass --> Inguinal mass
leg pain --> leg pain
hip

In [27]:

datasets_list = [dataset_kaggle, dataset_symbi, dataset_columbia]

# Padronizar os nomes de colunas (sintomas), removendo espaços e colocando tudo em minúsculo
for i in range(len(datasets_list)):
    df = datasets_list[i].copy()
    df.columns = [col.strip().lower() for col in df.columns]
    datasets_list[i] = df

# Coletar todos os sintomas únicos (ignorando coluna 'diseases')
all_symptoms = set()
for df in datasets_list:
    all_symptoms.update(df.columns[1:])  # Ignora a primeira coluna 'diseases'

# Reordenar e preencher todas as bases
normalized_datasets = []

for original_df in datasets_list:
    df = original_df.copy(deep=True)
    df.columns = [col.strip().lower() for col in df.columns]
    df = df.loc[:, ~df.columns.duplicated()]

    # Identifica colunas faltantes
    missing_cols = sorted(list(all_symptoms - set(df.columns[1:])))  # exclui 'diseases'

    # Cria um DataFrame com colunas faltantes preenchidas com 0
    if missing_cols:
        zeros_df = pd.DataFrame(0, index=df.index, columns=missing_cols)
        df = pd.concat([df, zeros_df], axis=1)

    # Reordena colunas
    ordered_columns = ["diseases"] + sorted(all_symptoms)
    df = df[ordered_columns]

    normalized_datasets.append(df)

# Concatenar os DataFrames
merged_dataset = pd.concat(normalized_datasets, ignore_index=True)

# Garantir que não há colunas duplicadas
merged_dataset = merged_dataset.loc[:, ~merged_dataset.columns.duplicated()]

# Mostrar info final
print(merged_dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263609 entries, 0 to 263608
Columns: 837 entries, diseases to yellowing of eyes
dtypes: int64(836), object(1)
memory usage: 1.6+ GB
None


In [28]:
#dataset unido (falta enriquecer os dados)
merged_dataset.head()

,diseases,abdomen acute,abdomen distended,abdominal bloating,abdominal colic,abdominal pain,abdominal tenderness,abnormal appearing skin,abnormal appearing tongue,abnormal breathing sounds,...,wrinkles on skin,wrist lump or mass,wrist pain,wrist stiffness or tightness,wrist swelling,wrist weakness,yellow color,yellow crust ooze,yellow sputum,yellowing of eyes
0,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Enriquecimento dos Dados

Agora que as bases já foram unidas e tiveram suas características padronizadas, é importante enriquecer os dados para aumentar a eficácia de treinamento do modelo a posteriori. Em primeiro momento, será adicionada uma coluna para a descrição da doença (de acordo com o UMLS) e outra coluna descrevendo possíveis fatores contribuintes para aquele caso, além dos sintomas em si.

In [29]:
#esta função retorna a descrição da doença de acordo com a UMLS
def get_condiction_description(name):
    doc = nlp(name)

    if not doc.ents:
        return ''  #retorna o nome original se nenhuma entidade for detectada

    entity = doc.ents[0]  #pega a primeira entidade detectada

    #acessar o linker
    linker = nlp.get_pipe("scispacy_linker")

    #se a entidade tem links no UMLS, retorna o nome do conceito mais relevante
    if hasattr(entity._, "kb_ents") and entity._.kb_ents:
        umls_id = entity._.kb_ents[0][0]  #pega o UMLS ID de maior score
        disease_entity = linker.kb.cui_to_entity[umls_id] #pega a entidade da condição no umls (Unified Medical Language System)
        disease_definition = disease_entity.definition
        return disease_definition #retorna a descrição da doença para substituir na base

    return ''  #se não houver links, retorna vazio

#testando a função
get_condiction_description("Myocardial Infarction")

'NECROSIS of the MYOCARDIUM caused by an obstruction of the blood supply to the heart (CORONARY CIRCULATION).'

In [30]:
#esta função serve para enriquecer os dados com fatores de risco para cada doença
def generate_risk_factors(disease):
    client = OpenAI(api_key="MINHA CHAVE AQUI")
    prompt = f"""
      List the main risk factors that contribute to the development of {disease}.
      Include genetic, environmental, lifestyle, and exposure to external agents.
    """
    response = client.completions.create(model="gpt-4o", prompt=prompt, max_tokens=200)
    return response.choices[0].text.strip()

In [31]:
#enriquecendo os dados
all_diseases = merged_dataset['diseases'].unique()

#padronizar os nomes das doenças
description_mapping = {disease: get_condiction_description(disease) for disease in all_diseases}

merged_dataset['diseases_description'] = merged_dataset['diseases'].map(description_mapping).fillna('There is no description')
merged_dataset.head()

C:\Users\mario\AppData\Local\Temp\ipykernel_19276\3457900238.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_dataset['diseases_description'] = merged_dataset['diseases'].map(description_mapping).fillna('There is no description')


,diseases,abdomen acute,abdomen distended,abdominal bloating,abdominal colic,abdominal pain,abdominal tenderness,abnormal appearing skin,abnormal appearing tongue,abnormal breathing sounds,...,wrist lump or mass,wrist pain,wrist stiffness or tightness,wrist swelling,wrist weakness,yellow color,yellow crust ooze,yellow sputum,yellowing of eyes,diseases_description
0,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...
1,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...
2,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...
3,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...
4,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...


In [32]:
list(merged_dataset.columns)

['diseases',
 'abdomen acute',
 'abdomen distended',
 'abdominal bloating',
 'abdominal colic',
 'abdominal pain',
 'abdominal tenderness',
 'abnormal appearing skin',
 'abnormal appearing tongue',
 'abnormal breathing sounds',
 'abnormal involuntary movements',
 'abnormal menstruation',
 'abnormal movement of eyelid',
 'abnormal sensation',
 'abnormal size or shape of ear',
 'abnormally hard consistency',
 'abscess bacterial',
 'absence of menstruation',
 'absences finding',
 'abusing alcohol',
 'acetylcholinesterase',
 'ache all over',
 'acne or pimples',
 'adverse reactions',
 'agitation',
 'air fluid level',
 'alcohol binge episode',
 'alcoholic withdrawal symptoms',
 'allergic reaction',
 'altered sensorium',
 'ambidexterity',
 'angina pectoris',
 'ankle pain',
 'ankle stiffness or tightness',
 'ankle swelling',
 'ankle weakness',
 'anorexia',
 'anosmia',
 'antisocial behavior',
 'anxiety',
 'anxiety and nervousness',
 'aphagia',
 'apnea',
 'apyrexial',
 'arm cramps or spasms',
 '

#### Adicionando fatores de risco

A Mayo Clinic é uma das instituições médicas mais respeitadas e renomadas do mundo, reconhecida pela excelência em pesquisa clínica, atendimento hospitalar e educação médica. Com sede nos Estados Unidos, ela é frequentemente classificada entre os melhores hospitais globais e atua como referência em diagnósticos, tratamentos e diretrizes clínicas. Seu portal oficial, voltado tanto para o público geral quanto para profissionais da saúde, é uma fonte homologada e cuidadosamente revisada por especialistas, oferecendo informações confiáveis sobre doenças, sintomas, causas e, especialmente, fatores de risco associados a condições médicas. Para esta pesquisa, os dados extraídos da Mayo Clinic fornecem conteúdo clínico qualificado e padronizado, enriquecendo a base de dados ao acrescentar dimensões relevantes ao perfil de cada doença. A inclusão desses fatores de risco, oriundos de uma fonte tão criteriosa, fortalece o rigor científico do modelo proposto e contribui significativamente para o desenvolvimento de um sistema de diagnóstico mais realista, interpretável e baseado em evidências.

In [33]:
def buscar_pagina_detalhada_mayo(disease_name):
    base_url = "https://www.mayoclinic.org/diseases-conditions/"
    search_term = f'search-results?q={disease_name.lower().replace(" ", "+")}'
    search_url = f"{base_url}{search_term}"  # heurística
    
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    
    # Passo 1: acessar a página de busca
    res = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Passo 2: encontrar a lista de resultados
    result_list = soup.find("ul", class_="cmp-search-results__results-list")
    if not result_list:
        print(f"❌ Nenhuma lista encontrada para: {disease_name}")
        return None

    # Passo 3: pegar o primeiro <a> da lista
    first_item = result_list.find("li", class_="cmp-search-result")
    if not first_item:
        print(f"❌ Nenhum resultado encontrado para: {disease_name}")
        return None

    link_tag = first_item.find("a", href=True)
    if not link_tag:
        print(f"❌ Link não encontrado no primeiro item para: {disease_name}")
        return None

    detail_url = link_tag["href"]
    print(f"🔗 Navegando para: {detail_url}")

    # Passo 4: acessar a página de detalhes da condição
    res_detail = requests.get(detail_url, headers=headers)
    soup_detail = BeautifulSoup(res_detail.text, "html.parser")

     # Passo 5: buscar <section aria-labelledby="risk-factors">
    risk_section = soup_detail.find("section", attrs={"aria-labelledby": "risk-factors"})

    if not risk_section:
        # Pega seções com título "Risk factors"
        risk_section = soup_detail.find("h2", string=lambda s: s and "risk factors" in s.lower())
        
        if risk_section:
            content = []
            for sibling in risk_section.find_next_siblings():
                if sibling.name == "h2":
                    break
                content.append(sibling.get_text(strip=True))
            return {
                "disease": disease_name,
                "detail_url": detail_url,
                "risk_factors_text": " ".join(content)
            }
        
        return {
            "disease": disease_name,
            "detail_url": detail_url,
            "risk_factors_text": ""
        }

    # Passo 4: extrair <p> e <li> como texto
    paragraphs = risk_section.find_all("p")
    lists = risk_section.find_all("li")

    text_parts = [p.get_text(strip=True) for p in paragraphs]
    list_items = [li.get_text(strip=True) for li in lists]

    risk_text = " ".join(text_parts + list_items)

    return {
        "disease": disease_name,
        "detail_url": detail_url,
        "risk_factors_text": risk_text
    }


In [34]:
#Teste da busca com uma das doenças listadas na base de dados
dados = buscar_pagina_detalhada_mayo('Panic Disorder')
print(dados)

🔗 Navegando para: https://www.mayoclinic.org/diseases-conditions/panic-attacks/symptoms-causes/syc-20376021
{'disease': 'Panic Disorder', 'detail_url': 'https://www.mayoclinic.org/diseases-conditions/panic-attacks/symptoms-causes/syc-20376021', 'risk_factors_text': 'Symptoms of panic disorder often start in the late teens or early adulthood and affect more women than men. Factors that may increase the risk of developing panic attacks or panic disorder include: Family history of panic attacks or panic disorderMajor life stress, such as the death or serious illness of a loved oneA traumatic event, such as sexual assault or a serious accidentMajor changes in your life, such as a divorce or the addition of a babySmoking or excessive caffeine intakeHistory of childhood physical or sexual abuse'}


Com a função de web scrapping pronta, é preciso percorrer as doenças da base para extrair os fatores de risco existentes no site da Mayo Clinc, esses dados vão ajudar na contextualização geral do modelo LLM, deixando a base mais diversa e incrementando a capacidade de dianóstico.

In [35]:
#Separando todas as doenças da base para pesquisar os fatores de risco
all_diseases_merged = list(merged_dataset['diseases'].unique())
all_diseases_merged

['Panic Disorder',
 'Polyp of vocal cord',
 'Turner Syndrome',
 'Cryptorchidism',
 'poisoning due to ethylene glycol',
 'Postmenopausal atrophic vaginitis',
 'fracture of the hand',
 'cellulitis or abscess of mouth',
 'eye alignment disorder',
 'headache after lumbar puncture',
 'Pyloric Stenosis',
 'salivary gland disorder',
 'Juvenile osteochondrosis of tibial tubercle',
 'Traumatic AND/OR non-traumatic injury',
 'Metabolic Diseases',
 'Vaginitis',
 'Sick Sinus Syndrome',
 'tinnitus of unknown cause',
 'Glaucoma',
 'Eating Disorders',
 'Transient Ischemic Attack',
 'Pyelonephritis',
 'Rotator Cuff Injuries',
 'chronic pain disorder',
 'problem during pregnancy',
 'Liver and Intrahepatic Biliary Tract Carcinoma',
 'Atelectasis',
 'Choledocholithiasis',
 'Liver Cirrhosis',
 'Aortic Aneurysm, Thoracic',
 'Hematoma, Subdural',
 'Diabetic Retinopathy',
 'Fibromyalgia',
 'ischemia of the bowel',
 'fetal alcohol syndrome',
 'Peritonitis',
 'Pancreatitis, Acute',
 'Thrombophlebitis',
 'Asthm

In [36]:
#Vamos criar um dicionário em que a key é a doença e o value é o fator de risco extraído
dict_risk_factors = {}
for disease in all_diseases_merged:
    risk_factors = buscar_pagina_detalhada_mayo(disease)
    if risk_factors:
        dict_risk_factors[disease] = risk_factors.get('risk_factors_text', None)
print(dict_risk_factors)

🔗 Navegando para: https://www.mayoclinic.org/diseases-conditions/panic-attacks/symptoms-causes/syc-20376021
❌ Nenhuma lista encontrada para: Polyp of vocal cord
🔗 Navegando para: https://www.mayoclinic.org/diseases-conditions/turner-syndrome/symptoms-causes/syc-20360782
🔗 Navegando para: https://www.mayoclinic.org/diseases-conditions/undescended-testicle/symptoms-causes/syc-20351995
❌ Nenhuma lista encontrada para: poisoning due to ethylene glycol
❌ Nenhuma lista encontrada para: Postmenopausal atrophic vaginitis
🔗 Navegando para: https://www.mayoclinic.org/diseases-conditions/broken-hand/symptoms-causes/syc-20450240
❌ Nenhuma lista encontrada para: cellulitis or abscess of mouth
❌ Nenhuma lista encontrada para: eye alignment disorder
❌ Nenhuma lista encontrada para: headache after lumbar puncture
🔗 Navegando para: https://www.mayoclinic.org/diseases-conditions/pyloric-stenosis/symptoms-causes/syc-20351416
❌ Nenhuma lista encontrada para: salivary gland disorder
❌ Nenhuma lista encontr

In [37]:
#Atualizando a base com a nova coluna para fatores de risco
merged_dataset['disease_risk_factors'] = merged_dataset['diseases'].map(dict_risk_factors).fillna('There is no risk factors for this disease')
merged_dataset.head()

C:\Users\mario\AppData\Local\Temp\ipykernel_19276\4225781718.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_dataset['disease_risk_factors'] = merged_dataset['diseases'].map(dict_risk_factors).fillna('There is no risk factors for this disease')


,diseases,abdomen acute,abdomen distended,abdominal bloating,abdominal colic,abdominal pain,abdominal tenderness,abnormal appearing skin,abnormal appearing tongue,abnormal breathing sounds,...,wrist pain,wrist stiffness or tightness,wrist swelling,wrist weakness,yellow color,yellow crust ooze,yellow sputum,yellowing of eyes,diseases_description,disease_risk_factors
0,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...,Symptoms of panic disorder often start in the ...
1,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...,Symptoms of panic disorder often start in the ...
2,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...,Symptoms of panic disorder often start in the ...
3,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...,Symptoms of panic disorder often start in the ...
4,Panic Disorder,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,A type of anxiety disorder characterized by un...,Symptoms of panic disorder often start in the ...


In [46]:
#Verificando valores faltantes nas colunas de descrição e fatores de risco
print(f"Quantidade total de doenças: {len(merged_dataset['diseases'].unique())}")
print(f"Quantidade de doenças com descrição: {len(merged_dataset['diseases_description'].unique())}")
print(f"Quantidade de doenças com fatores de risco: {len(merged_dataset['disease_risk_factors'].unique())}")

Quantidade total de doenças: 854
Quantidade de doenças com descrição: 657
Quantidade de doenças com fatores de risco: 340


##### Completando Sinteticamente

Para 76,93% das doenças da base foi possível identificar uma descrição com nlp, enquanto que para apenas 39,81% foram identificados fatores de risco com o web scraping. De modo a completar essa quantidade será utilizada uma construção sintética dos dados para as doenças restantes. Isso irá pemitir completar a base e seguir para as próximas etapas.

## Salvando resultados e considerações finais

Durante o desenvolvimento do pipeline proposto, foi realizada a consolidação de três fontes distintas de dados: um dataset do Kaggle, o SymbiPredict e uma base acadêmica da Universidade de Columbia. Após o processo de unificação, limpeza, padronização semântica e enriquecimento, obtivemos uma base final com 854 doenças, 836 sintomas e 263609 casos mapeados. O dados estão estruturados e prontos para o fine-tuning de modelos de linguagem aplicados à saúde.

Esse resultado reforça a importância de um pipeline cuidadosamente elaborado, mesmo com quantidade moderada de dados. A curadoria e o enriquecimento semântico mostraram-se cruciais para a construção de dados mais ricos e informativos, potencializando a capacidade dos modelos em compreender contextos clínicos. Assim, evidencia-se que a qualidade e o contexto dos dados são tão determinantes quanto a arquitetura do modelo em tarefas de diagnóstico automatizado.

In [38]:
#resultado final com o datset padronizado e enriquecido
merged_dataset.to_csv('./datasets/merged_dataset.csv') #salvando o arquivo